In [44]:
import numpy as np
from math import pi, cos, sin, sqrt


def skew_symmetric(v):

    return np.array([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

def exp_twist(theta, twist):

    omega = np.array(twist[:3])  # Angular part
    v = np.array(twist[3:])     # Linear part
    
    if np.all(omega == 0):  # Pure translation
        R = np.eye(3)
        p = v * theta
    else:  # Screw motion
        omega_skew = skew_symmetric(omega)
        omega_norm = np.linalg.norm(omega)
        
        R = np.eye(3) + np.sin(theta) * omega_skew + (1 - np.cos(theta)) * np.dot(omega_skew, omega_skew)
        p = (np.eye(3) * theta + (1 - np.cos(theta)) * omega_skew + (theta - np.sin(theta)) * np.dot(omega_skew, omega_skew)).dot(v)
    
    p = p.reshape(3, 1)
    return np.vstack((np.hstack((R, p)), np.array([0, 0, 0, 1])))

def forward_kinematics_poe(joints, L =[5,11,0,5]):

#     twists = np.array([
#         [0, 0, 1, 0, 0, 0],
#         [0, -1, 0, L[0], 0, 0],
#         [0, 0, 0, 0, 0, 1],
#         [0, -1, 0, L[0] + L[1], 0, 0]
#     ])

#     M = np.array([
#         [1, 0, 0, -L[3]],
#         [0, 1, 0, 0],
#         [0, 0, 1, L[0] + L[1]],
#         [0, 0, 0, 1]
#     ])
    twists = np.array([
        [0, 0, 1, 0, 0, 0],
        [0, -1, 0, L[0], 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, -1, 0,L[0], 0, -L[1]]
    ])

    M = np.array([
        [1, 0, 0, L[1]+L[3]],
        [0, 1, 0, 0],
        [0, 0, 1, L[0]],
        [0, 0, 0, 1]
    ])
    AdT = np.eye(4)
    T = np.eye(4)
    for i in range(len(twists)):
        T = np.dot(T, exp_twist(joints[i], twists[i]))
        
        T=np.around(T, 4)
        AdT=volocity(T,i)
        AdT=np.dot(AdT,AdT)
        Jocobian(AdT,i)
#         print("\nT{}={}".format(i,T))
    
    T = np.dot(T, M)
    T=np.around(T, 4)
    return T



In [45]:
def volocity(T,i):

    AdT=compute_adjoint(T)
    AdT=np.around(AdT, 4)
    print("\nAd_T_{}\n{}".format(i,AdT))
    return AdT

def Jocobian(AdT,i):
    twists = np.array([
        [0, 0, 1, 0, 0, 0],
        [0, -1, 0, L[0], 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, -1, 0,L[0], 0, -L[1]]
    ])
    
    print("\nJ_{}:{}".format(i,np.around(np.dot(AdT,np.transpose(twists[i])),4)))
        



def compute_adjoint(g):
    # Extract the rotation matrix R (3x3) and translation vector p (3x1) from the transformation matrix g (4x4)
    R = g[0:3, 0:3]
    p = g[0:3, 3]

    # Construct the skew-symmetric matrix of p for the cross product operation
    p_skew = np.array([[0, -p[2], p[1]],
                       [p[2], 0, -p[0]],
                       [-p[1], p[0], 0]])
    
    # Construct the adjoint transformation matrix Ad_g (6x6)
    Ad_g = np.zeros((6, 6))
    Ad_g[0:3, 0:3] = R
    Ad_g[3:6, 3:6] = R
    Ad_g[3:6, 0:3] = p_skew.dot(R)

    return Ad_g




In [46]:

joints = [0,0,0,0]; L =[5,11,0,5]

T=forward_kinematics_poe(joints,L)
T=np.around(T, 4)
print(T)


Ad_T_0
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]

J_0:[0. 0. 1. 0. 0. 0.]

Ad_T_1
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]

J_1:[ 0. -1.  0.  5.  0.  0.]

Ad_T_2
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]

J_2:[0. 0. 0. 1. 0. 0.]

Ad_T_3
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]

J_3:[  0.  -1.   0.   5.   0. -11.]
[[ 1.  0.  0. 16.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  5.]
 [ 0.  0.  0.  1.]]


In [47]:
joints = [0,pi/4,1,-pi/4]; 
L =[5,11,0,5]

T=forward_kinematics_poe(joints,L)
T=np.around(T, 4)
print(T)


Ad_T_0
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]

J_0:[0. 0. 1. 0. 0. 0.]

Ad_T_1
[[ 0.7071  0.     -0.7071  0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.      0.    ]
 [ 0.7071  0.      0.7071  0.      0.      0.    ]
 [ 0.     -1.4645  0.      0.7071  0.     -0.7071]
 [-1.4644  0.     -3.5355  0.      1.      0.    ]
 [ 0.      3.5355  0.      0.7071  0.      0.7071]]

J_1:[ 0. -1.  0.  5.  0. -0.]

Ad_T_2
[[ 0.7071  0.     -0.7071  0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.      0.    ]
 [ 0.7071  0.      0.7071  0.      0.      0.    ]
 [ 0.     -2.1716  0.      0.7071  0.     -0.7071]
 [-1.4644  0.     -4.5355  0.      1.      0.    ]
 [ 0.      4.2426  0.      0.7071  0.      0.7071]]

J_2:[0. 0. 0. 0. 0. 1.]

Ad_T_3
[[ 1.      0.     -0.      0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.      0.    ]
 [ 0.      0.      1.      0.      0.      0